In [19]:
import intake
import xarray as xr
import warnings
import xesmf as xe
import matplotlib.pyplot as plt

In [20]:
cat = intake.open_esm_datastore("/home/enkf6/dstiller/CMIP6/catalog.json")
# cat = intake.open_esm_datastore("/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cmip6.json")

In [21]:
experiment = "past2k"
model = "MPI-ESM1-2-LR"

In [22]:
cat.df

,variable_id,table_id,source_id,experiment_id,member_id,grid_label,time_range,activity_id,institution_id,version,path,dcpp_init_year
0,pr,Amon,MPI-ESM1-2-LR,past2k,r1i1p1f1,gn,700101-702012,PMIP,MPI-M,v20210714,/home/enkf6/dstiller/CMIP6/PMIP/MPI-M/MPI-ESM1...,NaN
1,pr,Amon,MPI-ESM1-2-LR,past2k,r1i1p1f1,gn,702101-704012,PMIP,MPI-M,v20210714,/home/enkf6/dstiller/CMIP6/PMIP/MPI-M/MPI-ESM1...,NaN
2,pr,Amon,MPI-ESM1-2-LR,past2k,r1i1p1f1,gn,704101-706012,PMIP,MPI-M,v20210714,/home/enkf6/dstiller/CMIP6/PMIP/MPI-M/MPI-ESM1...,NaN
3,pr,Amon,MPI-ESM1-2-LR,past2k,r1i1p1f1,gn,706101-708012,PMIP,MPI-M,v20210714,/home/enkf6/dstiller/CMIP6/PMIP/MPI-M/MPI-ESM1...,NaN
4,pr,Amon,MPI-ESM1-2-LR,past2k,r1i1p1f1,gn,708101-710012,PMIP,MPI-M,v20210714,/home/enkf6/dstiller/CMIP6/PMIP/MPI-M/MPI-ESM1...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1018,zos,Omon,MPI-ESM1-2-LR,past2k,r1i1p1f1,gn,875101-877012,PMIP,MPI-M,v20210714,/home/enkf6/dstiller/CMIP6/PMIP/MPI-M/MPI-ESM1...,NaN
1019,zos,Omon,MPI-ESM1-2-LR,past2k,r1i1p1f1,gn,877101-879012,PMIP,MPI-M,v20210714,/home/enkf6/dstiller/CMIP6/PMIP/MPI-M/MPI-ESM1...,NaN
1020,zos,Omon,MPI-ESM1-2-LR,past2k,r1i1p1f1,gn,879101-881012,PMIP,MPI-M,v20210714,/home/enkf6/dstiller/CMIP6/PMIP/MPI-M/MPI-ESM1...,NaN
1021,zos,Omon,MPI-ESM1-2-LR,past2k,r1i1p1f1,gn,881101-883012,PMIP,MPI-M,v20210714,/home/enkf6/dstiller/CMIP6/PMIP/MPI-M/MPI-ESM1...,NaN


In [23]:
cat.search(
    experiment_id=experiment,
    source_id=model,
    member_id="r1i1p1f1",
    table_id="Amon",
    variable_id="psl",
).unique()

variable_id                                                        [psl]
table_id                                                          [Amon]
source_id                                                [MPI-ESM1-2-LR]
experiment_id                                                   [past2k]
member_id                                                     [r1i1p1f1]
grid_label                                                          [gn]
time_range             [700101-702012, 702101-704012, 704101-706012, ...
activity_id                                                       [PMIP]
institution_id                                                   [MPI-M]
version                                                      [v20210714]
path                   [/home/enkf6/dstiller/CMIP6/PMIP/MPI-M/MPI-ESM...
dcpp_init_year                                                        []
derived_variable_id                                                   []
dtype: object

In [24]:
dss = cat.search(
    experiment_id=experiment,
    source_id=model,
    member_id="r1i1p1f1",
    table_id="Amon",
    variable_id="psl",
    grid_label="gn",
    # time_range="185001-186912"
).to_dataset_dict(progressbar=False)
psl = next(iter(dss.values())).isel(time=0)
psl

/home/disk/p/dstiller/.local/software/miniconda3/envs/enspred/lib/python3.11/site-packages/xarray/coding/times.py:832: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/disk/p/dstiller/.local/software/miniconda3/envs/enspred/lib/python3.11/site-packages/xarray/coding/times.py:832: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/disk/p/dstiller/.local/software/miniconda3/envs/enspred/lib/python3.11/site-packages/xarray/coding/times.py:832: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dty

<xarray.Dataset>
Dimensions:         (bnds: 2, lat: 96, lon: 192, member_id: 1, dcpp_init_year: 1)
Coordinates:
    time            object 7001-01-16 12:00:00
    time_bnds       (bnds) object dask.array<chunksize=(2,), meta=np.ndarray>
  * lat             (lat) float64 -88.57 -86.72 -84.86 ... 84.86 86.72 88.57
    lat_bnds        (lat, bnds) float64 dask.array<chunksize=(96, 2), meta=np.ndarray>
  * lon             (lon) float64 0.0 1.875 3.75 5.625 ... 354.4 356.2 358.1
    lon_bnds        (lon, bnds) float64 dask.array<chunksize=(192, 2), meta=np.ndarray>
  * member_id       (member_id) object 'r1i1p1f1'
  * dcpp_init_year  (dcpp_init_year) float64 nan
Dimensions without coordinates: bnds
Data variables:
    psl             (member_id, dcpp_init_year, lat, lon) float32 dask.array<chunksize=(1, 1, 96, 192), meta=np.ndarray>
Attributes: (12/54)
    Conventions:                      CF-1.7 CMIP-6.2
    activity_id:                      PMIP
    branch_method:                    no parent
    branch_time_in_parent:            0.0
    contact:                          cmip6-mpi-esm@dkrz.de
    data_specs_version:               01.00.30
    ...                               ...
    intake_esm_attrs:grid_label:      gn
    intake_esm_attrs:activity_id:     PMIP
    intake_esm_attrs:institution_id:  MPI-M
    intake_esm_attrs:version:         v20210714
    intake_esm_attrs:_data_format_:   netcdf
    intake_esm_dataset_key:           PMIP.MPI-M.MPI-ESM1-2-LR.past2k.Amon.gn

In [25]:
dss = cat.search(
    experiment_id=experiment,
    source_id=model,
    member_id="r1i1p1f1",
    table_id="Amon",
    variable_id="tas",
    grid_label="gn",
    # time_range="185001-186912"
).to_dataset_dict(progressbar=False)
tas = next(iter(dss.values())).isel(time=0)
tas

/home/disk/p/dstiller/.local/software/miniconda3/envs/enspred/lib/python3.11/site-packages/xarray/coding/times.py:832: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/disk/p/dstiller/.local/software/miniconda3/envs/enspred/lib/python3.11/site-packages/xarray/coding/times.py:832: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/disk/p/dstiller/.local/software/miniconda3/envs/enspred/lib/python3.11/site-packages/xarray/coding/times.py:832: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dty

<xarray.Dataset>
Dimensions:         (bnds: 2, lat: 96, lon: 192, member_id: 1, dcpp_init_year: 1)
Coordinates:
    time            object 7001-01-16 12:00:00
    time_bnds       (bnds) object dask.array<chunksize=(2,), meta=np.ndarray>
  * lat             (lat) float64 -88.57 -86.72 -84.86 ... 84.86 86.72 88.57
    lat_bnds        (lat, bnds) float64 dask.array<chunksize=(96, 2), meta=np.ndarray>
  * lon             (lon) float64 0.0 1.875 3.75 5.625 ... 354.4 356.2 358.1
    lon_bnds        (lon, bnds) float64 dask.array<chunksize=(192, 2), meta=np.ndarray>
    height          float64 2.0
  * member_id       (member_id) object 'r1i1p1f1'
  * dcpp_init_year  (dcpp_init_year) float64 nan
Dimensions without coordinates: bnds
Data variables:
    tas             (member_id, dcpp_init_year, lat, lon) float32 dask.array<chunksize=(1, 1, 96, 192), meta=np.ndarray>
Attributes: (12/54)
    Conventions:                      CF-1.7 CMIP-6.2
    activity_id:                      PMIP
    branch_method:                    no parent
    branch_time_in_parent:            0.0
    contact:                          cmip6-mpi-esm@dkrz.de
    data_specs_version:               01.00.30
    ...                               ...
    intake_esm_attrs:grid_label:      gn
    intake_esm_attrs:activity_id:     PMIP
    intake_esm_attrs:institution_id:  MPI-M
    intake_esm_attrs:version:         v20210714
    intake_esm_attrs:_data_format_:   netcdf
    intake_esm_dataset_key:           PMIP.MPI-M.MPI-ESM1-2-LR.past2k.Amon.gn

In [26]:
cat.search(
    experiment_id=experiment,
    source_id=model,
    member_id="r1i1p1f1",
    table_id="Omon",
    variable_id="tos",
    grid_label="gn",
    # time_range="185001-186912"
)

,unique
variable_id,1
table_id,1
source_id,1
experiment_id,1
member_id,1
grid_label,1
time_range,93
activity_id,1
institution_id,1
version,1


In [27]:
dss = cat.search(
    experiment_id=experiment,
    source_id=model,
    member_id="r1i1p1f1",
    table_id="Omon",
    variable_id="tos",
    grid_label="gn",
    # time_range="185001-186912"
).to_dataset_dict(progressbar=False)
tos = next(iter(dss.values())).isel(time=0)
tos

/home/disk/p/dstiller/.local/software/miniconda3/envs/enspred/lib/python3.11/site-packages/xarray/coding/times.py:832: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/disk/p/dstiller/.local/software/miniconda3/envs/enspred/lib/python3.11/site-packages/xarray/coding/times.py:832: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/disk/p/dstiller/.local/software/miniconda3/envs/enspred/lib/python3.11/site-packages/xarray/coding/times.py:832: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dty

<xarray.Dataset>
Dimensions:             (bnds: 2, j: 220, i: 256, vertices: 4, member_id: 1,
                         dcpp_init_year: 1)
Coordinates:
    time                object 7001-01-16 12:00:00
    time_bnds           (bnds) object dask.array<chunksize=(2,), meta=np.ndarray>
  * j                   (j) int32 0 1 2 3 4 5 6 ... 213 214 215 216 217 218 219
  * i                   (i) int32 0 1 2 3 4 5 6 ... 249 250 251 252 253 254 255
    latitude            (j, i) float64 dask.array<chunksize=(220, 256), meta=np.ndarray>
    longitude           (j, i) float64 dask.array<chunksize=(220, 256), meta=np.ndarray>
    vertices_latitude   (j, i, vertices) float64 dask.array<chunksize=(220, 256, 4), meta=np.ndarray>
    vertices_longitude  (j, i, vertices) float64 dask.array<chunksize=(220, 256, 4), meta=np.ndarray>
  * member_id           (member_id) object 'r1i1p1f1'
  * dcpp_init_year      (dcpp_init_year) float64 nan
Dimensions without coordinates: bnds, vertices
Data variables:
    tos                 (member_id, dcpp_init_year, j, i) float32 dask.array<chunksize=(1, 1, 220, 256), meta=np.ndarray>
Attributes: (12/54)
    Conventions:                      CF-1.7 CMIP-6.2
    activity_id:                      PMIP
    branch_method:                    no parent
    branch_time_in_parent:            0.0
    contact:                          cmip6-mpi-esm@dkrz.de
    data_specs_version:               01.00.30
    ...                               ...
    intake_esm_attrs:grid_label:      gn
    intake_esm_attrs:activity_id:     PMIP
    intake_esm_attrs:institution_id:  MPI-M
    intake_esm_attrs:version:         v20210714
    intake_esm_attrs:_data_format_:   netcdf
    intake_esm_dataset_key:           PMIP.MPI-M.MPI-ESM1-2-LR.past2k.Omon.gn

In [28]:
plt.scatter(tos["lon_bnds"], tos["lat_bnds"], s=0.1, alpha=0.2)

KeyError: 'lon_bnds'

In [ ]:
regridder = xe.Regridder(
    tos, xe.util.grid_global(2, 2, lon1=359), "bilinear", ignore_degenerate=True
)

/glade/work/dstiller/conda-envs/enspred/lib/python3.11/site-packages/xesmf/backend.py:56: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')


In [ ]:
regridder(tos, keep_attrs=True)

<xarray.Dataset>
Dimensions:         (member_id: 1, dcpp_init_year: 1, y: 90, x: 180, d2: 2)
Coordinates:
    time            object 0001-01-15 13:00:00
    time_bnds       (d2) object dask.array<chunksize=(2,), meta=np.ndarray>
  * member_id       (member_id) object 'r1i1p1f1'
  * dcpp_init_year  (dcpp_init_year) float64 nan
    lon             (y, x) float64 0.0 2.0 4.0 6.0 ... 352.0 354.0 356.0 358.0
    lat             (y, x) float64 -89.0 -89.0 -89.0 -89.0 ... 89.0 89.0 89.0
Dimensions without coordinates: y, x, d2
Data variables:
    tos             (member_id, dcpp_init_year, y, x) float32 dask.array<chunksize=(1, 1, 90, 180), meta=np.ndarray>
Attributes: (12/57)
    Conventions:                      CF-1.7 CMIP-6.2
    activity_id:                      CMIP
    branch_method:                    standard
    branch_time_in_child:             10950.0
    branch_time_in_parent:            0.0
    case_id:                          1548
    ...                               ...
    intake_esm_attrs:variable_id:     tos
    intake_esm_attrs:grid_label:      gn
    intake_esm_attrs:version:         v20191120
    intake_esm_attrs:_data_format_:   netcdf
    intake_esm_dataset_key:           CMIP.NCAR.CESM2-FV2.piControl.Omon.gn
    regrid_method:                    bilinear

In [ ]:
tos

<xarray.Dataset>
Dimensions:         (member_id: 1, dcpp_init_year: 1, nlat: 384, nlon: 320,
                     d2: 2, vertices: 4)
Coordinates:
    lat             (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    lon             (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
  * nlat            (nlat) int32 1 2 3 4 5 6 7 8 ... 378 379 380 381 382 383 384
  * nlon            (nlon) int32 1 2 3 4 5 6 7 8 ... 314 315 316 317 318 319 320
    time            object 0001-01-15 13:00:00
    time_bnds       (d2) object dask.array<chunksize=(2,), meta=np.ndarray>
    lat_bnds        (nlat, nlon, vertices) float32 dask.array<chunksize=(384, 320, 4), meta=np.ndarray>
    lon_bnds        (nlat, nlon, vertices) float32 dask.array<chunksize=(384, 320, 4), meta=np.ndarray>
  * member_id       (member_id) object 'r1i1p1f1'
  * dcpp_init_year  (dcpp_init_year) float64 nan
Dimensions without coordinates: d2, vertices
Data variables:
    tos             (member_id, dcpp_init_year, nlat, nlon) float32 dask.array<chunksize=(1, 1, 384, 320), meta=np.ndarray>
Attributes: (12/56)
    Conventions:                      CF-1.7 CMIP-6.2
    activity_id:                      CMIP
    branch_method:                    standard
    branch_time_in_child:             10950.0
    branch_time_in_parent:            0.0
    case_id:                          1548
    ...                               ...
    intake_esm_attrs:table_id:        Omon
    intake_esm_attrs:variable_id:     tos
    intake_esm_attrs:grid_label:      gn
    intake_esm_attrs:version:         v20191120
    intake_esm_attrs:_data_format_:   netcdf
    intake_esm_dataset_key:           CMIP.NCAR.CESM2-FV2.piControl.Omon.gn